<a href="https://colab.research.google.com/github/jhashekhar/nlp-projects/blob/master/n_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
corpus = ['he is a king',
          'she is a good programmer',
          'Gagan is a chef',
          'london is capital of England',
          'Ottawa is capital of Canada']

def tokenize_corpus(corpus):
  tokens = [x.split() for x in corpus]
  return tokens 

tokenized_corpus = tokenize_corpus(corpus)
tokenized_corpus

[['he', 'is', 'a', 'king'],
 ['she', 'is', 'a', 'good', 'programmer'],
 ['Gagan', 'is', 'a', 'chef'],
 ['london', 'is', 'capital', 'of', 'England'],
 ['Ottawa', 'is', 'capital', 'of', 'Canada']]

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [13]:
word_to_idx = {"hello": 0, "world": 1}
# 2 words in vocab and 5 dimensional embeddings
# word embeddings are a representation of the semantics of a word, efficiently
# encoding semantic information that might be relevant to the task at hand.
embeds = nn.Embedding(2, 5)
lookup_tensor = torch.tensor([word_to_idx["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [20]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow, 
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# we should tokenize the input, but we will ignore that for now
# build a list of tuples. Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i+1]], test_sentence[i+2])
            for i in range(len(test_sentence) - 2)]

print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [0]:
vocab = set(test_sentence)
word_to_idx = {word: i for i, word in enumerate(vocab)}

In [0]:
class NGramLanguageModeler(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_size):
    super().__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.fc1 = nn.Linear(context_size * embedding_dim, 128)
    self.fc2 = nn.Linear(128, vocab_size)
  
  def forward(self, inputs):
    embeds = self.embeddings(inputs).view((1, -1))
    out = F.relu(self.fc1(embeds))
    out = self.fc2(out)
    log_probs = F.log_softmax(out, dim=1)
    return log_probs

criterion = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [37]:
losses = []

for epoch in range(10):
  total_loss = 0.0
  for context, target in trigrams:

    context_idxs = torch.tensor([word_to_idx[w] for w in context], dtype=torch.long)
    model.zero_grad()
    log_probs = model(context_idxs)
    loss = criterion(log_probs, torch.tensor([word_to_idx[target]], dtype=torch.long))
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  losses.append(total_loss)
  
print(losses)

[160.93199169635773, 158.75936841964722, 156.60798907279968, 154.4789187014103, 152.37201884388924, 150.28672191500664, 148.22367069125175, 146.18378630280495, 144.16628995537758, 142.1713998913765]
